## 스마트 포인터 조합하기
smart pointer는 한계가 있는 것처럼 보이지만, 조합해서 사용하면 매우 강력해질 수 있습니다.

Rc<Vec<Foo>> - heap에 있는 immutable한 데이터 구조의 동일한 vector를 대여할 수 있는 복수의 smart pointer를 복제할 수 있게 해줍니다.

Rc<RefCell<Foo>> - 복수의 smart pointer가 동일한 Foo struct를 mutable/immutable하게 대여할 수 있게 해줍니다.

Arc<Mutex<Foo>> - 복수의 smart pointer가 임시의 mutable/immutable한 대여를 CPU 쓰레드 독점 방식으로 잠글 수 있게 해줍니다.

메모리 상세:

이런 조합이 많이 포함된 주제를 확인할 수 있을겁니다. 내부 데이터를 변경하기 위해 immutable한 데이터 유형(복수의 smart pointer가 소유할 수 있음)을 사용합니다. 이를 Rust에서는 "내부 가변성" 패턴이라고 합니다. 
이는 Rust의 컴파일 타임 체크와 동일 수준의 안전성으로 런타임의 메모리 사용 규칙을 변경할 수 있는 패턴입니다.

In [2]:
use std::cell::RefCell;
use std::rc::Rc;

struct Pie {
    slices: u8,
}

impl Pie {
    fn eat_slice(&mut self, name: &str) {
        println!("{}가 한 조각 먹었습니다!", name);
        self.slices -= 1;
    }
}

struct SeaCreature {
    name: String,
    pie: Rc<RefCell<Pie>>,
}

impl SeaCreature {
    fn eat(&self) {
        // mutable 대여를 위해 파이에 대한 smart pointer를 사용
        let mut p = self.pie.borrow_mut();
        // 한 입 먹자!
        p.eat_slice(&self.name);
    }
}

fn main() {
    let pie = Rc::new(RefCell::new(Pie { slices: 8 }));
    // ferris와 sarah에겐 파이에 대한 smart pointer의 복제가 주어집니다
    let ferris = SeaCreature {
        name: String::from("ferris"),
        pie: pie.clone(),
    };
    let sarah = SeaCreature {
        name: String::from("sarah"),
        pie: pie.clone(),
    };
    ferris.eat();
    sarah.eat();

    let p = pie.borrow();
    println!("{} 조각 남았습니다", p.slices);
}


